### Load tensorflow

In [222]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
# this allows plots to appear directly in the notebook
%matplotlib inline
import keras 
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.optimizers import SGD 
from keras import utils 
import numpy as np 
from scipy import stats

In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [6]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [8]:
data.drop(['date', 'symbol'], axis=1, inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [16]:
df_prices = data.iloc[:1000]

In [17]:
df_prices.shape

(1000, 5)

In [18]:
scaler = StandardScaler()
prices_col = df_prices.columns
df_prices = scaler.fit_transform(df_prices)
df_prices = pd.DataFrame(df_prices, columns=prices_col)

In [20]:
df_prices.head(10)

,open,close,low,high,volume
0,0.972495,1.012559,0.973227,1.003826,-0.217909
1,1.003498,0.912099,0.932280,0.991731,-0.202500
2,0.851734,0.825867,0.845721,0.892928,-0.195369
3,0.836318,0.854497,0.821015,0.853748,-0.228789
4,0.862526,0.826210,0.831208,0.851874,-0.270128
5,0.836832,0.836153,0.838292,0.830239,-0.291610
6,0.853104,0.789866,0.805292,0.847445,-0.301874
7,0.802574,0.816095,0.761408,0.812693,-0.313238
8,0.799490,0.784380,0.793716,0.810138,-0.291908
9,0.805143,0.747521,0.758298,0.827003,-0.262181


### Divide the data into train and test sets

In [21]:
X = df_prices.drop(['close'], axis=1)
y = df_prices['close']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

## Building the graph in tensorflow

1.Define input data placeholders

In [85]:
n_samples = 700
n_features  =4
learning_rate = 0.01

In [129]:
X_in = tf.placeholder(tf.float32, [None, n_features], "X_in") 
y_in = tf.placeholder(tf.float32, [None], "y_in") 

2.Define Weights and Bias

In [199]:
# Set model weights and bias 

W = tf.Variable(tf.random_normal([n_features,1]), name="W")
b = tf.Variable(np.random.randn(), name="b")   

3.Prediction

In [200]:
# Construct a linear model 
h = tf.add(tf.matmul(X_in, W), b)      # can use matmul function too = tf.matmul(X, W) + b

4.Loss (Cost) Function [Mean square error]

In [201]:
n_samples = X_train.shape[0]

In [202]:
# Mean squared error 
cost = tf.reduce_mean(tf.square(tf.subtract(y_in, h)))     #(loss func)

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [203]:
# Gradient descent 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # opearational tensor

## Execute the Graph for 100 epochs and observe the loss

In [204]:
# Initializing the variables 
init = tf.global_variables_initializer()

In [205]:
training_epochs = 100

In [206]:
# Launch the graph 
with tf.Session() as sess: 
	# Load initialized variables in current session 
	sess.run(init) 

	# Fit all training data 
	for epoch in range(training_epochs): 

		# perform gradient descent step 
		sess.run(optimizer, feed_dict={X_in: X_train, y_in: y_train}) 
		
		# Display logs per epoch step 
		if (epoch+1) % 10 == 0: 
			c = sess.run(cost, feed_dict={X_in: X_train, y_in: y_train}) 
			print("Epoch:{0:6} \t Cost:{1:10.4}". 
				format(epoch+1, c)) 
			
	# Print final parameter values 
	print("Optimization Finished!") 
	training_cost = sess.run(cost, feed_dict={X_in: X_train, y_in: y_train}) 
	print("Final training cost:", training_cost, "W:", sess.run(W), "b:", 
		sess.run(b), '\n') 
	
	# Testing the model 
	testing_cost = sess.run(tf.reduce_mean(tf.square(tf.subtract(y_in, h))), 
							feed_dict={X_in: X_test, y_in: y_test}) 
	
	print("Final testing cost:", testing_cost) 
	lossdiff = abs(training_cost - testing_cost)
	print("Absolute mean square loss difference:", lossdiff) 
	Weight = sess.run(W)
	Bias = sess.run(b)


Epoch:    10 	 Cost:     3.567
Epoch:    20 	 Cost:     2.414
Epoch:    30 	 Cost:     1.875
Epoch:    40 	 Cost:      1.58
Epoch:    50 	 Cost:     1.402
Epoch:    60 	 Cost:      1.29
Epoch:    70 	 Cost:     1.218
Epoch:    80 	 Cost:     1.171
Epoch:    90 	 Cost:     1.141
Epoch:   100 	 Cost:      1.12
Optimization Finished!
Final training cost: 1.1204737 W: [[-2.6616518 ]
 [ 1.6774362 ]
 [ 0.9815274 ]
 [ 0.03951158]] b: 0.22093138 

Final testing cost: 0.88917863
Absolute mean square loss difference: 0.23129511


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [193]:
print(Weight.shape)

(4,)


In [194]:
print(Bias.shape)

()


### Find the Absolute mean square loss difference between training and testing loss.

In [157]:
print(lossdiff)

0.24467516


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [210]:
model = Sequential() # the first layer has to specify the dimensions of the input vector 
model.add( Dense( units = 4, input_shape =( 4,))) 
model.add( Dense( units = 1)) 
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [211]:
model.compile(loss='mean_squared_error', optimizer='sgd')

### Execute the model

In [217]:
# train the model 
model.fit(X_train, y_train, epochs = 100) 

# evaluate the model and print the accuracy score 

loss = model.evaluate(X_test, y_test) 

print('\ n loss:', loss) 


Epoch 1/100
700/700 [==============================] - 0s 36us/step - loss: 1.3808e-04
Epoch 2/100
700/700 [==============================] - 0s 27us/step - loss: 1.3804e-04
Epoch 3/100
700/700 [==============================] - 0s 36us/step - loss: 1.3767e-04
Epoch 4/100
700/700 [==============================] - 0s 50us/step - loss: 1.3816e-04
Epoch 5/100
700/700 [==============================] - 0s 43us/step - loss: 1.3818e-04
Epoch 6/100
700/700 [==============================] - 0s 64us/step - loss: 1.3786e-04
Epoch 7/100
700/700 [==============================] - 0s 33us/step - loss: 1.3801e-04
Epoch 8/100
700/700 [==============================] - 0s 41us/step - loss: 1.3812e-04
Epoch 9/100
700/700 [==============================] - 0s 46us/step - loss: 1.3803e-04
Epoch 10/100
700/700 [==============================] - 0s 46us/step - loss: 1.3812e-04
Epoch 11/100
700/700 [==============================] - 0s 47us/step - loss: 1.3830e-04
Epoch 12/100
700/700 [===================

700/700 [==============================] - 0s 41us/step - loss: 1.3778e-04
Epoch 94/100
700/700 [==============================] - 0s 44us/step - loss: 1.3719e-04
Epoch 95/100
700/700 [==============================] - 0s 57us/step - loss: 1.3782e-04
Epoch 96/100
700/700 [==============================] - 0s 60us/step - loss: 1.3752e-04
Epoch 97/100
700/700 [==============================] - 0s 51us/step - loss: 1.3795e-04
Epoch 98/100
700/700 [==============================] - 0s 60us/step - loss: 1.3747e-04
Epoch 99/100
700/700 [==============================] - 0s 53us/step - loss: 1.3799e-04
Epoch 100/100
300/300 [==============================] - 0s 30us/step
\ n loss: 0.0001372928974402991


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [264]:
data_iris = pd.read_csv('./iris.csv')

In [265]:
data_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [276]:
X_i = data_iris.drop(['Id','Species'], axis=1)
Y_i = data_iris['Species']

In [277]:
X_i = stats.zscore(X_i, axis = 1)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [278]:
X_i.shape

(150, 4)

In [279]:
Y_i = pd.get_dummies(Y_i)

### Divide the dataset into Training and test (70:30)

In [280]:
Xi_train, Xi_test, yi_train, yi_test = train_test_split(X_i, Y_i, test_size=0.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [281]:
model = Sequential() # the first layer has to specify the dimensions of the input vector 
model.add( Dense( units = 10, activation ='sigmoid', input_shape =( 4,))) # add dropout layer for preventing overfitting 
model.add( Dense( units = 8, activation ='sigmoid')) 
model.add( Dense( units = 3, activation ='softmax')) # print the summary of our model 
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 27        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


### Fitting the model and predicting 

In [283]:
model.compile( loss ='categorical_crossentropy', optimizer = SGD(), metrics =['accuracy']) 

# train the model 
model.fit(Xi_train, yi_train, epochs = 100) 

Epoch 1/100
105/105 [==============================] - 0s 2ms/step - loss: 1.1583 - acc: 0.3619
Epoch 2/100
105/105 [==============================] - 0s 76us/step - loss: 1.1507 - acc: 0.3619
Epoch 3/100
105/105 [==============================] - 0s 85us/step - loss: 1.1448 - acc: 0.3619
Epoch 4/100
105/105 [==============================] - 0s 76us/step - loss: 1.1400 - acc: 0.3619
Epoch 5/100
105/105 [==============================] - 0s 95us/step - loss: 1.1356 - acc: 0.3619
Epoch 6/100
105/105 [==============================] - 0s 95us/step - loss: 1.1317 - acc: 0.3619
Epoch 7/100
105/105 [==============================] - 0s 85us/step - loss: 1.1267 - acc: 0.3619
Epoch 8/100
105/105 [==============================] - 0s 76us/step - loss: 1.1235 - acc: 0.3619
Epoch 9/100
105/105 [==============================] - 0s 86us/step - loss: 1.1202 - acc: 0.3619
Epoch 10/100
105/105 [==============================] - 0s 66us/step - loss: 1.1177 - acc: 0.3619
Epoch 11/100
105/105 [========

105/105 [==============================] - 0s 104us/step - loss: 1.0826 - acc: 0.3619
Epoch 85/100
105/105 [==============================] - 0s 95us/step - loss: 1.0822 - acc: 0.3619
Epoch 86/100
105/105 [==============================] - 0s 95us/step - loss: 1.0821 - acc: 0.3619
Epoch 87/100
105/105 [==============================] - 0s 95us/step - loss: 1.0819 - acc: 0.3619
Epoch 88/100
105/105 [==============================] - 0s 95us/step - loss: 1.0817 - acc: 0.3619
Epoch 89/100
105/105 [==============================] - 0s 323us/step - loss: 1.0818 - acc: 0.3619
Epoch 90/100
105/105 [==============================] - 0s 95us/step - loss: 1.0817 - acc: 0.3619
Epoch 91/100
105/105 [==============================] - 0s 85us/step - loss: 1.0816 - acc: 0.3619
Epoch 92/100
105/105 [==============================] - 0s 104us/step - loss: 1.0812 - acc: 0.3619
Epoch 93/100
105/105 [==============================] - 0s 85us/step - loss: 1.0810 - acc: 0.3619
Epoch 94/100
105/105 [========

In [255]:
X_i

array([[-0.689485  ,  1.59026379,  0.70060572, -0.46707048, -1.13431403],
       [-0.18928292,  1.64045201,  0.44166016, -0.56784877, -1.32498047],
       [ 0.33143213,  1.41496023,  0.45890602, -0.75209598, -1.4532024 ],
       [ 0.81400877,  1.18401276,  0.25900279, -0.72767451, -1.52934982],
       [ 1.01208567,  1.01208567,  0.28916733, -0.84684719, -1.46649148],
       [ 1.17834225,  0.89778457,  0.19639038, -0.83232112, -1.44019609],
       [ 1.54685328,  0.53252326,  0.02535825, -0.81991677, -1.28481803],
       [ 1.60303038,  0.50506437, -0.08051751, -0.77589598, -1.25168125],
       [ 1.77370608,  0.26834668, -0.22253139, -0.71340945, -1.10611191],
       [ 1.76923689,  0.28517305, -0.23861419, -0.70420284, -1.11159291],
       [ 1.76093253,  0.27580871, -0.17503245, -0.75847395, -1.10323484],
       [ 1.84981106,  0.09735848, -0.24339619, -0.68150934, -1.02226401],
       [ 1.87656507,  0.07471102, -0.32081792, -0.67239919, -0.95805898],
       [ 1.91181327, -0.0402487 , -0.3

### Report Accuracy of the predicted values

In [0]:
scores = model.evaluate( x_test, y_test) 

print('\ n loss:', scores[ 0]) 

print('\ n accuracy:', scores[ 1])